In [811]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.agents import AgentExecutor, create_json_chat_agent
from langchain.prompts import PromptTemplate
from langchain_core.documents.base import Document
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool, StructuredTool, Tool

from dotenv import load_dotenv
from vertexai.generative_models import (
    SafetySetting,
    HarmCategory,
    HarmBlockThreshold,
)

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [812]:
messages = []

def convert_to_str(messages):
    ans = ""
    for i in messages:
        ans += i[0] + " : " + i[1] + "\n"

    return ans

In [813]:
safety = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_UNSPECIFIED,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_NONE
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_NONE
    )
]

In [814]:
model = ChatGoogleGenerativeAI(
    model="gemini-pro",
    client=genai,
    temperature=0.5,
    safety_settings=None
)

embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001")

new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

user_data = """
  
    ```
       [{
  "phone_no": 932489237,
  "name": "Raj Patel",
  "address": {
  "apartment_no": "223, Suryasthali Appartment",
  "area_street": "Manavta Nagar",
  "landmark": "Hanuman Mandir",
  "town_city": "Bengaluru",
  "state": "Karnataka",
  "pincode": 530068
  },
  "email": "raja23@gmail.com",
  "subscriptionStatus": false,
  "previousOrders": [
    {
  "order_id": “20234435843-1107”,
  "status": "Delivered",
  "transaction": {
  "transaction_id": "txn123dsafasd",
  "status": "Successful",
  "payment_method": "Amazon Pay",
  "total_amount": 3000,
  "timestamp": {
    "$date": "2024-06-14T18:55:34.443Z"
  }
  },
  "items": [
  "product_id": 3247387,
  "name": "HRX Oversized T-Shirt",
  "description": "Cotton-Comfy fit Oversized T-Shirt",
  "category": "Clothing-Men-TShirt",
  "average_rating": 3,
  "price": 399,
  "reviews": [
    "Very Comfortable and Affordable",
    "Cheap and affordable",
    "Don't BUY AT ALLLL"
  ]
  ],
    ```
"""

first_context = """
    Your name is Radhika from Amazon Customer Support Agent Team and you will be helping a customer today. 

    You will be speaking, so output text accordingly. 

    Do not verify anything about user because he is already verified.

    You can ask user if you need any details.
    
    You should never ask them to contact Amazon Customer service, instead if you are unable to find the answer, forward the call to agent and terminate by [AGENT]
"""

initial_prompt_template = """
    {first_context}

    {context}

    User Question: 
    Only try to answer this and add nothing else.
    {user_input}

    User Data:
    Only relevant to queries which needs user data to be checked!
    {user_data}

    Previous Chat History:
    While thinking of answer, go through the conversation to get the context of what user mean, but answer only according to User Question
    {chat_history}

    WRITE [TERMINATE] IF THE USER IS SATISFIED
    IF UNABLE TO FIND THE USER ANSWER IN CONTEXT, TRANSFER THE CALL TO AGENT BY [AGENT]

    {ending_lines}
"""

prompt = PromptTemplate(
    input_variables=["context", "user_input", "first_context", "user_data", "ending_lines", "chat_history"],
    template=initial_prompt_template
)

chain = load_qa_chain(llm=model, chain_type="stuff", prompt = prompt)


In [815]:
new_chat_template = """
    Your name is Radhika from Amazon Customer Support Agent Team and you will be helping a customer today. 
    You will be speaking, so output text accordingly. 
    Do not verify anything about user because he is already verified.

    You can ask user if you need any details.
    
    {tools}
    {tool_names}
    {agent_scratchpad}
"""

new_prompt = PromptTemplate(
    input_variables=["tools", "tool_names", "agent_scratchpad"],
    template=new_chat_template
)

In [816]:
agent = create_json_chat_agent(model, tools=new_tools, prompt=new_prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools=new_tools) # Add this line.

agent_executor.invoke({"input": "Thank you for the help!"})

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: Hi there, my name is Radhika. I'm from Amazon Customer Support. How can I help you today?

In [817]:
def filter_contexts(term):
    contexts = new_db.similarity_search_with_score(term, k=3)

    ans_contexts = []

    for i in contexts:
        print(i[1])
        if(i[1] > 0.5):
            ans_contexts.append(i[0])

    return ans_contexts

In [818]:
#First Input
user_question = " Hi, I received a damaged item and I’d like to request a refund."
context_user = filter_contexts(user_question)

first_message = initial_prompt_template.format(first_context = first_context, input_documents = context_user, user_input = user_question, user_data = user_data, ending_lines = "Your call is now live with the agent! Do not include anything of previous chat in replies. Use them to only generate future answers.", context = '', chat_history = '')

res = chain({
    "first_context": first_context, 
    "input_documents": context_user, 
    "user_input":  user_question, 
    "user_data": "User Data \n" + user_data, 
    "ending_lines": "Begin! Call is live with customer", 
    "chat_history": convert_to_str(messages)
}, return_only_outputs=True)

messages.append(("system", first_message))
messages.append(("agent", res['output_text']))
res

0.40284258
0.4272114
0.44770235


{'output_text': 'I am sorry to hear that you received a damaged item. I can help you with that. May I know the order ID of the damaged item?'}

In [821]:
model.invoke("")

[('system',
  '\n    \n    Your name is Radhika from Amazon Customer Support Agent Team and you will be helping a customer today. \n\n    You will be speaking, so output text accordingly. \n\n    Do not verify anything about user because he is already verified.\n\n    You can ask user if you need any details.\n    \n    You should never ask them to contact Amazon Customer service, instead if you are unable to find the answer, forward the call to agent and terminate by [AGENT]\n\n\n    \n\n    User Question: \n    Only try to answer this and add nothing else.\n     Hi, I received a damaged item and I’d like to request a refund.\n\n    User Data:\n    Only relevant to queries which needs user data to be checked!\n    \n  \n    ```\n       [{\n  "phone_no": 932489237,\n  "name": "Raj Patel",\n  "address": {\n  "apartment_no": "223, Suryasthali Appartment",\n  "area_street": "Manavta Nagar",\n  "landmark": "Hanuman Mandir",\n  "town_city": "Bengaluru",\n  "state": "Karnataka",\n  "pincode"

In [819]:
user_question = "Sure, can you check the recent order. It is an HRX Oversized T-Shirt"
context_user = filter_contexts(user_question)

response = chain({
    "first_context": "", 
    "input_documents": context_user, 
    "user_input": user_question, 
    "user_data": user_data, 
    "ending_lines": "", 
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))
response

0.7050432
0.71106863
0.7267091


{'output_text': '[AGENT]'}

In [651]:
user_question = "Okay, can you tell me something about me!"
context_user = new_db.similarity_search(user_question)

response = chain({
    "first_context": "",
    "input_documents": "",
    "user_input":  user_question,
    "user_data": "", "ending_lines": "",
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))

In [652]:
response

{'output_text': '[TERMINATE]\nThank you for calling Amazon!'}

In [653]:
user_question = "Thank you for the answer!"
context_user = filter_contexts(user_question)

response = chain({
    "first_context": "",
    "input_documents": context_user,
    "user_input":  user_question,
    "user_data": "", "ending_lines": "",
    "chat_history": convert_to_str(messages)
}, return_only_outputs = True)

messages.append(("human", user_question))
messages.append(("agent", response['output_text']))

0.5054102
0.57055926
0.6090071


In [654]:
response

{'output_text': 'Are you satisfied with my service?\n[TERMINATE]\nThank you for calling Amazon!'}